In [13]:
import pandas as pd

"""TODO: change filename to Metadata file in project of interest"""
meta = pd.read_csv('Metadata KPNALF_Other.csv')
print("Number of Isolates = ", len(meta))

Number of Isolates =  907


In [18]:
"""TODO: change filename to Vitek file in project of interest"""
vt = pd.read_csv('Vitek KPNALF_Other.csv')
print("Number of Isolates = ", len(vt))

Number of Isolates =  907


In [6]:
""" Note: This section relates the names of the drugs in the Vitek results to the drug names in the database"""

vt_dict = {'Testing Date': 'vitek_date',
               'AM-Ampicillin': 'Ampicillin (AMP)',
               'AMC-Amoxicillin/Clavulanic Acid': 'Amoxicillin/Clavulanic (AUG)',
               'AN-Amikacin': 'Amikacin (AMK)',
               'CAZ-Ceftazidime': 'Ceftazidime (CAZ)',
               'CIP-Ciprofloxacin': 'Ciprofloxacin (CIP)',
               'CRO-Ceftriaxone': 'Ceftriaxone (CRO)',
               'CZ-Cefazolin': 'Cefazolin (CFZ)',
               'FEP-Cefepime': 'Cefepime (FEP)',
               'FOX-Cefoxitin': 'Cefoxitin (CFX)',
               'FT-Nitrofurantoin': 'Nitrofurantoin (NIT) ',
               'GM-Gentamicin': 'Gentamicin (GEN)',
               'MEM-Meropenem': 'Meropenem (MEM)',
               'NOR-Norfloxacin': 'Norfloxacin (NOR)',
               'SXT-Trimethoprim/Sulfamethoxazole': 'Trimethoprim/Sulfamethoxazole (SXT)',
               'TCC-Ticarcillin/Clavulanic Acid': 'Ticarcillin/Clavulanic acid (TIM)',
               'TM-Tobramycin': 'Tobramycin (TOB)',
               'TMP-Trimethoprim': 'Trimethoprim (TMP)',
               'TZP-Piperacillin/Tazobactam': 'Piperacillin/Tazobactam (PTZ)',
               'P-Benzylpenicillin': 'Benzylpenicillin',
               'OXSF-Cefoxitin Screen': 'Cefoxitin Screen',
               'CM-Clindamycin': 'Clindamycin',
               'DAP-Daptomycin': 'Daptomycin',
               'E-Erythromycin': 'Erythromycin',
               'FA-Fusidic Acid': 'Fusidic Acid',
               'ICR-Inducible Clindamycin Resistance': 'Inducable Clindamycin Resistance',
               'LNZ-Linezolid': 'Linezolid',
               'MUP-Mupirocin': 'Mupirocin',
               'OX1-Oxacillin': 'Oxacillin',
               'RA-Rifampicin': 'Rifampicin',
               'TEC-Teicoplanin': 'Teicoplanin',
               'TE-Tetracycline': 'Tetracycline',
               'VA-Vancomycin': 'Vancomycin'
               }



33501


In [ ]:
""" Note: The organism dictionary is important!! Abbreviated names in vitek results do not match 
            names in the database and finding the correct results when multiple swabs/bugs are detected
            can become difficult/impossible
"""
org_dict = pd.read_excel('organism_dictionary_finalised.xlsx')
org_dict['Vitek name 1'] = org_dict['Vitek name 1'].str.lstrip()
org_dict['Vitek name 2'] = org_dict['Vitek name 2'].str.lstrip()
org_dict['Vitek name 3'] = org_dict['Vitek name 3'].str.lstrip()
org_dict['Vitek name 4'] = org_dict['Vitek name 4'].str.lstrip()


In [ ]:
""" Read most recent library file with Vitek results"""
df_vitek = pd.read_excel('Vitek_Library_2017_Jun2021.xlsx')
print(len(df_vitek))

In [19]:
""" This function does the actual mapping from vitek results to the database format """

def map_vitek_template(df_vitek, meta, vitek, org_dict, vt_dict, outfile):
    vt_template = vitek

    print("Beginning to search and assign vitek records")
    for i, row in vt_template.iterrows():
        if row['data'] == 'Yes' or row['data'] == 'Yes updated':
            continue
        
#         print(row['tube_code'])
        acc = meta.loc[(meta['tube_code'] == row['tube_code']), 'ID_number']
        """ Accession number if needed to find the vitek result"""
        if acc.isnull().any():
            vt_template.loc[i, 'data'] = 'No'
            print(row['tube_code'], 'no accession number')
            continue
        
        acn = acc.values[0]
        acn = int(acn.replace('-', ''))
        t_acn_match = df_vitek.loc[(df_vitek['Lab ID'] == acn)]
        ##first look for an matching accession number
        if len(t_acn_match) == 0:
            vt_template.loc[i, 'data'] = 'No'
            print(row['tube_code'], "No matching found in vitek records")
            continue
        
        org_name = meta.loc[(meta['tube_code'] == row['tube_code']), 'species_reported'].values[0]
        org_genus = org_name.split()[0]
        """ Find match to exact organism but also just the genus"""
        match_organisms = org_dict.loc[org_dict['IsolName'].str.match(org_name), :].dropna(axis=1).values.tolist()
        match_genus = org_dict.loc[org_dict['IsolName'].str.match(org_genus), :].dropna(axis=1).values.tolist()

        """ if you find nothing, you're shit outta luck"""
        if len(match_organisms) == 0 and len(match_genus) == 0:
            if len(t_acn_match) > 0:
                print(row['tube_code'], org_name, ' no match found in org dictionary')
                print("But there is an ACN match", t_acn_match.loc[:, 'Organism Name'])
            continue

#         print("Orgs len = ", match_organisms)
#         print("genus = ", match_genus)
        """ let's see if there are exact matches or just matches to genus"""
        t_exact_match = []
        t_genus_match = []
        if len(match_organisms) > 0:
            match_organisms = match_organisms[0][1:]
            t_exact_match = df_vitek.loc[(df_vitek['Lab ID'] == acn) &
                                         (df_vitek['Organism Name'].isin(list(match_organisms)))]
        if len(match_genus) > 0:
            match_genus = match_genus[0][1:]
            t_genus_match = df_vitek.loc[(df_vitek['Lab ID'] == acn) &
                                         (df_vitek['Organism Name'].isin(list(match_genus)))]
#             print(t_genus_match)

        if len(t_exact_match) == 0 and len(t_genus_match) == 0:
            # no record found
            vt_template.loc[i, 'data'] = 'No'
            print(acn, 'no record found')
            continue
        elif len(t_exact_match) == 1:
            print(row['tube_code'], " one record found")
            vt_template.loc[i, 'data'] = 'Yes'
            vt_template.loc[i, 'vitek_run'] = 'Alfred'
            for j in vt_dict.keys():
                if j == 'Testing Date':
                    vt_template.loc[i, vt_dict[j]] = t_exact_match.loc[:, j].dt.strftime('%Y-%m-%d').values[0]
                else:
                    vt_template.loc[i, vt_dict[j]] = t_exact_match.loc[:, j].values[0]
        elif len(t_genus_match) == 1:
            print(row['tube_code'], " one record found")
            vt_template.loc[i, 'data'] = 'Yes'
            vt_template.loc[i, 'vitek_run'] = 'Alfred'
            for j in vt_dict.keys():
                vt_template.loc[i, vt_dict[j]] = t_genus_match.loc[:, j].values[0]
        elif len(t_exact_match) > 1:
            print(row['tube_code'], " > 1 record found")
            """ find the index where Testing Date is max"""
            key_index = t_exact_match.index[t_exact_match['Testing Date'].argmax()]

            vt_template.loc[i, 'data'] = 'Yes Updated'
            vt_template.loc[i, 'vitek_run'] = 'Alfred'
            for j in vt_dict.keys():
                if j.find('Testing Date') > -1:
                    vt_template.loc[i, vt_dict[j]] = t_exact_match.loc[key_index, j].strftime('%Y-%m-%d')
                else:
                    vt_template.loc[i, vt_dict[j]] = t_exact_match.loc[key_index, j]
        elif len(t_genus_match) >= 1:
            print(row['tube_code'], " > 1 record found")
            key_index = t_genus_match.index[t_genus_match['Testing Date'].argmax()]
            """ find the index where Testing Date is a match"""
            vt_template.loc[i, 'data'] = 'Yes Updated'
            vt_template.loc[i, 'vitek_run'] = 'Alfred'
            for j in vt_dict.keys():
                if j.find('Testing Date') > -1:
                    vt_template.loc[i, vt_dict[j]] = t_genus_match.loc[key_index, j].strftime('%Y-%m-%d')
                else:
                    vt_template.loc[i, vt_dict[j]] = t_genus_match.loc[key_index, j]
    print("Done.")
    vt_template.to_csv(outfile, index=False)
    return vt_template


""" TODO: Remember to change the name of the output file!"""
outfile_vt = 'Vitek_KPNALF_Other' + 'temp' + '.csv'
vt_template = map_vitek_template(df_vitek, meta, vt, org_dict, 
                                 vt_dict, outfile_vt)